# ENF Extraction from Audio and Video Files

Audio recordings may contain sum hum that is caused by the grid frequency interfering with the audio signal. If this noise is present in the audio signal depends on the recording equipment, the cabling, and so on.

It is known that the grid frequency is not stricty 50 or 60 Hz but slightly fluctuates around the nominal value. These fluctuations are then also present in the audio audio recordings. If one matches the fluctuation of in the audio with the fluctuations of the grid frequency in the past that is is possible to chronolocate the audio recording, that is, determine tha time when the recording was made.

For this matching to work, one needs:
- access to a database of historical network frequencies,
- an audio clip containing a sufficient amount of network noise.

## Import Standard Modules

In [ ]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import scipy as sc

!# Install the Python modules that are not yet present on Colab
try:
  import py7zr
except:
  !pip install py7zr
  import py7zr

## Load Custom ENF Modules from Github

In [ ]:
!# Clone the files on github to Colab so that they can be used
![ -d enf-matching ] || git clone https://github.com/CoRoe/enf-matching.git

# Add the path of the just cloned Python files to the Python path:
if not '/content/enf-matching' in sys.path:
    sys.path.insert(0, '/content/enf-matching')
#print(sys.path)

from enf import AudioClipEnf
from enf import notch_filter, butter_bandpass_filter

In [ ]:
# @title Choose an audio or video file to analyse

# TODO: The current mechanism is akward. Check
# https://colab.research.google.com/github/NeuromatchAcademy/course-content-dl/blob/main/projects/ComputerVision/spectrogram_analysis.ipynb
# for ideas.

filename = "enf-matching/samplemedia/001.wav" # @param {"type":"string","placeholder":"Audio or video file"}

clip = AudioClipEnf()
if clip.loadAudioFile(filename):
  print(f"Loaded '{filename}' ok, sample rate {clip.sampleRate()}")
else:
  print(f"Failed to load audio file '{filename}'")

## Spectrogram

This step displays the spectrogram of the input file without any filtering. The brighter the colour the stronger the frequency component.

In [ ]:
NFFT = 2048
fig, (ax2) = plt.subplots(nrows=1, sharex=True)
Pxx, freqs, bins, im = ax2.specgram(clip.data, NFFT=NFFT, Fs=clip.sampleRate())
# The `specgram` method returns 4 objects. They are:
# - Pxx: the periodogram
# - freqs: the frequency vector
# - bins: the centers of the time bins
# - im: the .image.AxesImage instance representing the data in the plot
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Frequency (Hz)')
ax2.set_title('Spectrogram')

In [ ]:
# @title For the next steps, some parameters have to be chosen.
grid_freq = "50" # @param ["50","60"]
harmonic = "2" # @param ["1","2"]


# Spectrogram of Filtered Data

In [ ]:
# See
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.iirpeak.html#scipy.signal.iirpeak

filter_quality = 20 # Filter quality

# FIXME: Output of the notch filter is not plausible and does not agree
# with the STFT result.
#filtered_data = notch_filter(clip.data, int(grid_freq), clip.sampleRate(),
#                             filter_quality)

b, a = sc.signal.iircomb(int(grid_freq), filter_quality, ftype='peak',
                         fs=clip.sampleRate(), pass_zero=True)

# Frequency response
freq, h = sc.signal.freqz(b, a, fs=clip.sampleRate())
print("freq:", freq[-1])

# Filter the data
response = abs(h)

# To avoid divide by zero when graphing
response[response == 0] = 1e-20

# Apply the filter
filtered_data = sc.signal.filtfilt(b, a, clip.data)

# Plot frequency response and spectrogram
fig, ax = plt.subplots(2, 1)
ax[0].plot(freq, 20*np.log10(abs(response)), color='blue')
ax[0].set_title("Frequency Response")
ax[0].set_xlabel("Frequency (Hz)")
ax[0].set_ylabel("Amplitude (dB)")
#ax[0].set_xlim([0, 100])
ax[0].set_ylim([-30, 10])
ax[0].grid(True)

NFFT = 2048
Pxx, freqs, bins, im = ax[1].specgram(filtered_data, NFFT=NFFT, Fs=clip.sampleRate())
# The `specgram` method returns 4 objects. They are:
# - Pxx: the periodogram
# - freqs: the frequency vector
# - bins: the centers of the time bins
# - im: the .image.AxesImage instance representing the data in the plot
ax[1].set_xlabel('Time (s)')
ax[1].set_ylabel('Frequency (Hz)')
ax[1].set_title('Spectrogram')

# Spectrogram after Bandpass

In [ ]:
butter_order = 20
NFFT = 4096
freq_band = 0.200

locut = int(harmonic) * (int(grid_freq) - freq_band)
hicut = int(harmonic) * (int(grid_freq) + freq_band)
ylim_lower = int(harmonic) * (int(grid_freq) - 5 * freq_band)
ylim_upper = int(harmonic) * (int(grid_freq) + 5 * freq_band)

filtered_data = butter_bandpass_filter(clip.data, locut, hicut,
                                        clip.sampleRate(), butter_order)
t = np.linspace(0, len(filtered_data)/clip.sampleRate(), len(filtered_data))

fig, ax = plt.subplots(1, 1)
#print(filtered_data.shape)
#print(t.shape)

Pxx, freqs, bins, im = ax.specgram(filtered_data, NFFT=NFFT, Fs=clip.sampleRate())
# The `specgram` method returns 4 objects. They are:
# - Pxx: the periodogram
# - freqs: the frequency vector
# - bins: the centers of the time bins
# - im: the .image.AxesImage instance representing the data in the plot
ax.set_ylim((ylim_lower, ylim_upper))
ax.set_xlabel('Time (s)')
ax.set_ylabel('Frequency (Hz)')
ax.set_title('Spectrogram')

# ENF over Time

This step determines the variation of the ENF signal over time.

In [ ]:
clip.makeEnf(int(grid_freq), 0.200, int(harmonic))
t, f_enf = clip.getEnf()
fig, (ax1) = plt.subplots(nrows=1, sharex=True)
ax1.plot(t, f_enf/1000)
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('ENF (Hz)')